In [50]:
import MeCab
import itertools
import pprint
import math

In [51]:
m = MeCab.Tagger("mecab-ipadic-neologd")
line = m.parse("アプリ登録しても電気使用量が「-」となっていて、計測値が見れません。")
line= line.split('\n')
[l.split('\t') for l in line]

[['アプリ', '名詞,一般,*,*,*,*,*'],
 ['登録', '名詞,サ変接続,*,*,*,*,登録,トウロク,トーロク'],
 ['し', '動詞,自立,*,*,サ変・スル,連用形,する,シ,シ'],
 ['て', '助詞,接続助詞,*,*,*,*,て,テ,テ'],
 ['も', '助詞,係助詞,*,*,*,*,も,モ,モ'],
 ['電気', '名詞,一般,*,*,*,*,電気,デンキ,デンキ'],
 ['使用', '名詞,サ変接続,*,*,*,*,使用,シヨウ,シヨー'],
 ['量', '名詞,接尾,一般,*,*,*,量,リョウ,リョウ'],
 ['が', '助詞,格助詞,一般,*,*,*,が,ガ,ガ'],
 ['「', '記号,括弧開,*,*,*,*,「,「,「'],
 ['-」', '名詞,サ変接続,*,*,*,*,*'],
 ['と', '助詞,格助詞,一般,*,*,*,と,ト,ト'],
 ['なっ', '動詞,自立,*,*,五段・ラ行,連用タ接続,なる,ナッ,ナッ'],
 ['て', '助詞,接続助詞,*,*,*,*,て,テ,テ'],
 ['い', '動詞,非自立,*,*,一段,連用形,いる,イ,イ'],
 ['て', '助詞,接続助詞,*,*,*,*,て,テ,テ'],
 ['、', '記号,読点,*,*,*,*,、,、,、'],
 ['計測', '名詞,サ変接続,*,*,*,*,計測,ケイソク,ケイソク'],
 ['値', '名詞,接尾,一般,*,*,*,値,チ,チ'],
 ['が', '助詞,格助詞,一般,*,*,*,が,ガ,ガ'],
 ['見れ', '動詞,自立,*,*,一段,連用形,見れる,ミレ,ミレ'],
 ['ませ', '助動詞,*,*,*,特殊・マス,未然形,ます,マセ,マセ'],
 ['ん', '助動詞,*,*,*,不変化型,基本形,ん,ン,ン'],
 ['。', '記号,句点,*,*,*,*,。,。,。'],
 ['EOS'],
 ['']]

In [65]:
class wordvec_lib:
    
    def _word_attrs(self, txt):
        m = MeCab.Tagger("mecab-ipadic-neologd")
        line = m.parse(txt).split('\n')[:-2]
        return [l.split('\t')[1].split(',') for l in line]

    def _filter_words(self, word_attrs):
        # filter noun and verb
        w = word_attrs
        w = [m for m in w if m[0] in ['名詞', '動詞']]
        return w

    def _attr_to_word_vec(self, attrs):
        words = [a[6] for a in attrs]
        return {g[0] : len(list(g[1])) for g in itertools.groupby(sorted(words))}

    def get_word_vec(self, txt):
        attrs = self._word_attrs(txt)
        attrs = self._filter_words(attrs)
        return self._attr_to_word_vec(attrs)
    
    def vec_normalize(self, v):
        veclen = math.sqrt(sum([v*v for (k, v) in v.items()]))
        return {k: v/veclen for (k, v) in v.items()}
    
    def dot_product(self, v1, v2):
        return sum(v1[key]*v2.get(key, 0) for key in v1)
    
    def cosine_similarity(self, v1, v2):
        v1, v2 = self.vec_normalize(v1), self.vec_normalize(v2)
        return self.dot_product(v1, v2)
    
    def manhattan_distance(self, v1, v2):
        return sum

In [64]:
w = wordvec_lib()
vec_q = w.get_word_vec('私は駅に向かって向かいます')
w.vec_normalize(vec_q)

{'向かう': 0.8164965809277261, '私': 0.4082482904638631, '駅': 0.4082482904638631}

In [62]:
a = {'hoge': 2, 'fefe': 3}
{k: v/2.0 for (k, v) in a.items()}

{'fefe': 1.5, 'hoge': 1.0}

In [66]:
class qa:
    
    w = wordvec_lib()
    
    def set_standard_qa(self, qa_list):
        self.qa_list = qa_list
        self.vec_db = [self.w.get_word_vec(q[0]) for q in qa_list]
        pp = pprint.PrettyPrinter(indent=4)
        print('=== word count vector ===')
        pp.pprint(self.vec_db)

    def q(self, q_txt):
        self.w = wordvec_lib()
        vec_q = self.w.get_word_vec(q_txt)
        inner_prods = [self.w.cosine_similarity(v, vec_q) for v in self.vec_db]
        max_idx = inner_prods.index(max(inner_prods))
        return self.qa_list[max_idx][1]

FAQ情報ソース: 
<a href="http://sumai.panasonic.jp/hems/shs/handbook169/faq.html">スマートHEMS よくあるご質問</a>

In [67]:
qa_txt = [('アプリ登録しても電気使用量が「-」となっていて、計測値が見れません。', '計測値の取得はアプリ登録後最大30分かかります。'),
          ('本アプリで何が見える化できますか？', '代表的なものとして、電気使用量、電気料金、太陽光やエネファーム発電量・電力収支(発電設備導入家庭のみ)、CO2排出量などが見える化できます。'),
          ('プッシュ通知されるサービス内容を教えてください。', '家全体の電気の使いすぎのお知らせ、季節に合わせた省エネアドバイス、目標設定チャレンジ、洗濯機の運転終了情報、エアコンの長時間運転情報、AiSEGのシステムエラー・アップデート、アプリのアップデートなどのご連絡を行います。'),
          ('タイムラインに残る履歴は何件くらいですか？', '約200件です。'),
          ('回路別電気料金は何日分さかのぼって確認することができますか？', '昨日と今日の2日分です。'),
          ('省エネアドバイスで配信するメッセージは何種類ありますか？', '2016年6月現在、下記の2種類を予定しています。 •家電の使い方に関する節電アイデア•家電に限定せず暮らし全般に関する節電アイデア'),
          ('ランキング比較対象の世帯はどの様に決めるのですか？'),('AiSEGのサーバーサービス（無料）に登録している世帯で、本アプリの利用者を対象にしています。またオール電化／ガス使用の家庭別に分けております。今後は、世帯別・地域別も検討いたします。'),
          ('本アプリで電気使用量などのデータはどの位の期間確認できますか？'),('電気使用量に関する閲覧できるデータは下記の通りです。1日ごとで表示した場合：先々月、先月、今月の本日まで　1時間ごとで表示した場合：6日前から本日まで'),
          ('本アプリで表示される金額は、どのように計算された値ですか？'),('AiSEG本体に登録した料金単価を利用時間帯に応じて計算した値です。'),
          ('太陽光の全量買取の時の発電表示はできますか？'),('全量買取時の発電量は表示できません。'),
          ('本アプリで見れる計測データは、リアルタイムの値ですか？'),('瞬時値ではございません。計測されたデータは30分ごとにAiSEGからインターネット経由でセンターサーバーにアップロードされます。ですので30分ごとに更新された値が本アプリで確認できます。'),
          ('本アプリの料金表示は、電力会社の料金プランに合わせ自動的に更新されますか？'),('対応しておりません。'),
          ('使いすぎチェックの色区分はどのような基準になっていますか？'),('次の通りです。青……省エネ状態(過去平均と比較して80％未満)黄……注意(80-100％未満) 赤……使いすぎ(100％以上)'),
          ('CO2排出量は何を元に算出されていますか？'),('CO2排出の基準値はパナソニックで定めた基準値です。（0.430kgCO2/kWh）'),
          ('本アプリで表示される金額は電力会社の請求額と同じですか？'),('本アプリに表示される電気使用量や電気料金は目安です。電力事業者から請求される電気料金とは異なります。'),
          ('アプリから通知が届かないのですが。'),('通知が届かない場合は以下の3つをご確認ください。お手持ちのスマートフォンが通信できる状態にしてください。 お手持ちのスマートフォンの設定で本アプリの通知をONにしてください。アプリのメニューにある通知設定から、必要な機能の通知をONにしてください。'),
          ('AiSEG本体で単価を変更してもアプリの電気料金、単価が変更されません。'),('単価の変更はアプリに反映するまでに最長2日かかります。'),
          ('「おうちの電気使用情報が取得できていません」という通知について'),('「おうちの電気使用情報が取得できていません」のメッセージは、AiSEGとサーバが通信できなくなった場合に通知されます。'),
          ('本アプリの動作中にエラーが表示されましたが、どうすればいいですか？'),('以下をご確認ください。通信経路上の遅延等で、一時的なエラーが発生することがあります。再度操作をしてください。■スマートHEMSに関するお問い合わせ〔住宅エネルギーマネージメントシステム　ご相談窓口〕フリーダイヤル　0120-081-701〔受付時間〕365日／9:00～20:00※携帯電話・PHSからもご利用になれます。●上記番号がご利用できない場合　06-6780-2099'),
          ('エアコンのつけ方は？',      'エアコンをつけるにはボタン２を押します'),
          ('エアコンの温度の上げ方は？', 'エアコンの温度を上げるにはボタン４を押します'),
          ('テレビのつけ方は？',        'テレビをつけるにはボタン３を押します')]
qa_elec = qa()
qa_elec.set_standard_qa(qa_txt)

=== word count vector ===
[   {   '*': 2,
        'いる': 1,
        'する': 1,
        'なる': 1,
        '使用': 1,
        '値': 1,
        '登録': 1,
        '見れる': 1,
        '計測': 1,
        '量': 1,
        '電気': 1},
    {'*': 1, 'できる': 1, '何': 1, '化': 1, '見える': 1},
    {   'くださる': 1,
        'する': 1,
        'れる': 1,
        'サービス': 1,
        'プッシュ': 1,
        '内容': 1,
        '教える': 1,
        '通知': 1},
    {'タイム': 1, 'ライン': 1, '件': 1, '何': 1, '履歴': 1, '残る': 1},
    {   'こと': 1,
        'さかのぼる': 1,
        'する': 1,
        'できる': 1,
        '何': 1,
        '分': 1,
        '別': 1,
        '回路': 1,
        '料金': 1,
        '日': 1,
        '確認': 1,
        '電気': 1},
    {   'ある': 1,
        'する': 1,
        'アドバイス': 1,
        'メッセージ': 1,
        '何': 1,
        '省エネ': 1,
        '種類': 1,
        '配信': 1},
    {'ラ': 1},
    {'*': 1},
    {'本': 1},
    {'電': 1},
    {'本': 1},
    {'*': 1},
    {'太': 1},
    {'全': 1},
    {'本': 1},
    {'瞬': 1},
    {'本': 1},
    {'対': 1},
    {'使': 1},
    

In [68]:
qa_elec.q('エアコンの温度設定はどうやるの？')

'エアコンの温度を上げるにはボタン４を押します'

In [69]:
qa_elec.q('エアコンの温度設定はどうやるの？')

'エアコンの温度を上げるにはボタン４を押します'

In [70]:
qa_elec.q('エアコンをつけたいんだけど')

'エアコンをつけるにはボタン２を押します'

In [71]:
qa_elec.q('エアコンをつけたいんだけど')

'エアコンをつけるにはボタン２を押します'

In [72]:
qa_elec.q('テレビをつけるには？')

'テレビをつけるにはボタン３を押します'

In [73]:
qa_elec.q('テレビをつけるには？')

'テレビをつけるにはボタン３を押します'

In [74]:
qa_elec.q('どうやって照明をつけるの？')

'計測値の取得はアプリ登録後最大30分かかります。'

In [75]:
qa_elec.q('どうやって照明をつけるの？')

'計測値の取得はアプリ登録後最大30分かかります。'

In [76]:
qa_elec.q('タイムラインに残る履歴は何件くらいですか')

'約200件です。'

In [77]:
qa_elec.q('タイムラインに残る履歴は何件くらいですか')

'約200件です。'

In [78]:
qa_elec.q('アプリで計測値を見るには？')

'計測値の取得はアプリ登録後最大30分かかります。'

In [79]:
qa_elec.q('アプリで計測値を見るには？')

'計測値の取得はアプリ登録後最大30分かかります。'

In [80]:
qa_elec.q('省エネアドバイス')

'2016年6月現在、下記の2種類を予定しています。 •家電の使い方に関する節電アイデア•家電に限定せず暮らし全般に関する節電アイデア'